In [32]:
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from shutil import which
from fake_useragent import UserAgent
import time
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from datetime import datetime

given_date_str = "2022-08-10"
given_date = datetime.strptime(given_date_str, '%Y-%m-%d')
sellers = []
url = "https://www.bybit.com/copyTrade/trade-center/detail?leaderMark=ZIFBeYBreTZKNwkw/7Dobg=="

In [15]:
def readPastTrades(url):
    ua = UserAgent()
    user_agent = ua.random

    # Import options

    # Create Options object
    chrome_options = Options()
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--start-maximized");

    # Add argument to Options to use headless browser
    # chrome_options.add_argument("--headless")

    chrome_path = which("chromedriver")

    # Add options argument to Chrome driver to use headless browser
    driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)

    past_trades_pages = []

    driver.get(url)

    # search_input = driver.find_element_by_id("search_form_input_homepage")
    # search_input.send_keys("My User Agent")
    # search_input.send_keys(Keys.ENTER)

    # Print the html markup
    time.sleep(1)

    while True:
        try:
            past_trades_pages.append(driver.page_source)

            next_button = driver.find_element_by_xpath(
                "//li[@class='ant-pagination-next' and @aria-disabled='false']/button")
            next_button.click()
            time.sleep(0.8)


        except NoSuchElementException:
            break

        except Exception as e:
            print(e)
            break

    # Close the driver
    driver.close()
    print(len(past_trades_pages))
    return past_trades_pages


In [16]:
past_trades_pages = readPastTrades(url)
count = 0
while len(past_trades_pages) < 2:
    if count == 5:
        break
    past_trades_pages = readPastTrades(url)
    count += 1

1
37


In [17]:
from math import ceil


def readCopyTrades(url):
    ua = UserAgent()
    user_agent = ua.random

    # Create Options object
    chrome_options = Options()
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--start-maximized")

    # Add argument to Options to use headless browser
    # chrome_options.add_argument("--headless")

    chrome_path = which("chromedriver")
    driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)

    driver.get(url)
    time.sleep(1)
    copy_trade_button = driver.find_element_by_xpath("(//div[@class='ant-tabs-tab-btn'])[2]")
    copy_pages = ceil(int(copy_trade_button.text.strip().split()[2][1:-1]) / 20)
    time.sleep(1)
    copy_trade_button.click()
    time.sleep(1)

    copy_trades_pages = []

    while True:
        try:
            copy_trades_pages.append(driver.page_source.encode('utf-8'))

            next_button = driver.find_element_by_xpath(
                "(//li[@class='ant-pagination-next' and @aria-disabled='false']/button)[2]")

            # next_button = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "((//li[@class='ant-pagination-next' and @aria-disabled='false']/button)[2]")))

            next_button.click()
            time.sleep(1)


        except NoSuchElementException:
            break

    print(len(copy_trades_pages))
    driver.close()

    return (copy_trades_pages, copy_pages)



In [18]:
copy_trades_pages, copy_pages = readCopyTrades(url)
count = 0
while (len(copy_trades_pages) < copy_pages) or copy_pages == 0:
    if count == 5:
        break
    copy_trades_pages, copy_pages = readCopyTrades(url)
    count += 1




1
3


In [19]:
# profits = 0
# losses = 0
percentage = 0

done = False
for copy_trades_page in copy_trades_pages:

    if done:
        break
    soup = BeautifulSoup(
        copy_trades_page)  # If this line causes an error, run 'pip install html5lib' or install html5lib
    # print(soup.prettify())

    table = soup.findAll("tbody", {"class": "ant-table-tbody"})[1]
    rows = table.findAll("tr", {"class": "ant-table-row ant-table-row-level-0"})

    for row in rows:
        # print(row)
        tds = row.findAll("td", {"class": "ant-table-cell"})
        outcome = float(tds[4].find("span").text.strip().split("%")[0])
        date_str = tds[3].text.strip()
        leverage = int(tds[0].text.strip().split()[1].strip().split("x")[0].strip())
        # print(leverage, outcome)
        date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

        if given_date > date:
            done = True
            break

        # print(date)
        # print(outcome)
        # if outcome[0] == "+":
        #     profits += 1
        #
        # elif outcome[0] == "-":
        #     losses += 1

        percentage += outcome / leverage
        # print(outcome/leverage)

print(percentage)

    # print(profits, losses)

-433.39


In [33]:
# profits = 0
# losses = 0

done = False
for past_trades_page in past_trades_pages:

    if done:
        break

    soup = BeautifulSoup(
        past_trades_page)  # If this line causes an error, run 'pip install html5lib' or install html5lib
    # print(soup.prettify())

    table = soup.findAll("tbody", {"class": "ant-table-tbody"})[0]
    rows = table.findAll("tr", {"class": "ant-table-row ant-table-row-level-0"})

    for row in rows:
        tds = row.findAll("td", {"class": "ant-table-cell"})
        outcome = float(tds[5].find("span").text.strip().split("%")[0])
        date_str = tds[4].text.strip()
        leverage = int(tds[0].text.strip().split()[1].strip().split("x")[0].strip())

        date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

        if given_date > date:
            done = True
            break

        # print(date)
        # print(outcome)

        percentage += outcome / leverage
        # print(outcome/leverage)
        # if outcome[0] == "+":
        #     profits += 1
        #
        # elif outcome[0] == "-":
        #     losses += 1

sellers.append([url, round(percentage, 2)])

In [34]:
print(sellers)

[['https://www.bybit.com/copyTrade/trade-center/detail?leaderMark=ZIFBeYBreTZKNwkw/7Dobg==', 3050.26]]
